In [2]:
import pandas as pd
import re
from konlpy.tag import Okt
from collections import defaultdict

df = pd.read_csv("Rdata.csv")

okt = Okt()

def extract_keywords(text):
    if pd.isnull(text):
        return []
    text = re.sub(r"[^\uAC00-\uD7A3\s]", "", str(text))
    return [word for word, tag in okt.pos(text) if tag in ["Noun", "Adjective"]]

df["keywords"] = df["review"].apply(extract_keywords)

# 3. 테마 키워드 사전 정의 (UX 표현 포함)
theme_keywords = {
    "☕ 커피 향 가득한 카페": [
        "진하다", "깊다", "고소하다", "향기롭다", "향", "바디감", "농도", "묵직하다", "원두",
        "핸드드립", "에스프레소", "로스팅", "싱글오리진", "추출", "드립", "콜드브루",
        "플랫화이트", "전문적", "수준", "퀄리티", "전문점", "정성"
    ],
    "🍰 디저트 천국": [
        "케이크", "디저트", "마카롱", "초코", "크림", "빵", "와플", "젤라또", "타르트", "빙수"
    ],
    "💸 실속 있는 선택": [
        "가성", "가성비", "혜자", "합리적", "알찬", "푸짐", "구성", "가격대비", "저렴", "싸다", "자주", "단골",
        "테이크아웃", "빠르다", "편의성", "간단", "프렌차이즈", "체인점", "메가", "컴포즈", "이디야", "포즈", "빽"
    ],    
    "💻 집중하기 좋은 곳": [
        "공부", "노트북", "작업", "조용", "집중", "콘센트", "와이파이", "자리", "타이핑", "앉다"
    ],
    "❤️ 함께 오기 좋은 공간": [
        "데이트", "연인", "커플", "애인", "로맨틱", "기념일", "남자친구", "여자친구", "사진찍기좋다", "설레다", "분위기", "좋은시간", "오붓하다", "추억", "기억에남다",
        "달달하다", "감성", "감성있다", "따뜻하다", "분위기좋고", "함께"
    ]
}

# 4. 테마 분류 및 키워드 점수 카운트
theme_counts = defaultdict(lambda: defaultdict(int))

for idx, row in df.iterrows():
    cafe = row["카페이름"]
    keywords = row["keywords"]
    count_per_theme = {theme: 0 for theme in theme_keywords}
    for word in keywords:
        for theme, words in theme_keywords.items():
            if word in words:
                count_per_theme[theme] += 1
    max_theme = max(count_per_theme, key=count_per_theme.get)
    max_count = count_per_theme[max_theme]
    if max_count > 0:
        theme_counts[max_theme][cafe] += max_count

# 5. 상위 카페 추출 → 테마별 상위 5개
result = {}
for theme, cafes in theme_counts.items():
    ranked = sorted(cafes.items(), key=lambda x: -x[1])[:5]
    result[theme] = ranked

# 6. 예쁜 텍스트 형태로 출력
for theme, cafe_list in result.items():
    print(f"\n{theme}")
    for i, (cafe, score) in enumerate(cafe_list, 1):
        print(f"{i}. {cafe}")


🍰 디저트 천국
1. 릴리베이커리
2. 세이케이크
3. 카페 그자체 베이커리베이커리
4. 레스트인무드
5. 레브 디저트카페

💻 집중하기 좋은 곳
1. MouseRabbit카페
2. 카페온더플랜 건대점카페,디저트
3. 할리스 건대입구점카페
4. 커피마켓카페
5. 꿈꾸다917카페,디저트

💸 실속 있는 선택
1. 컴포즈커피 건대스타시티몰점카페
2. 컴포즈커피 건대양꼬치거리점
3. 컴포즈커피 자양신양초교점
4. 빽다방 광진화양삼거리점테이크아웃커피
5. 고망고 건대점

❤️ 함께 오기 좋은 공간
1. 플레어 에스프레소 바
2. 엠케이갤러리스튜디오
3. 브루크 성수점
4. 심리카페멘토 건대점
5. 구르다방

☕ 커피 향 가득한 카페
1. 리틀히포 로스팅랩카페,디저트
2. 에스프레소 바 시 자양점
3. 건대커피 랩
4. 최가커피 자양점
5. 코에오 COEO카페,디저트
